In [1]:
!pip install implicit rectools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.6 MB 220 kB/s 
     |████████████████████████████████| 89 kB 8.6 MB/s 
     |████████████████████████████████| 13.4 MB 56.9 MB/s 
     |████████████████████████████████| 310 kB 59.9 MB/s 
     |████████████████████████████████| 1.1 MB 46.2 MB/s 
     |████████████████████████████████| 60 kB 8.5 MB/s 
     |████████████████████████████████| 88 kB 8.0 MB/s 
     |████████████████████████████████| 188 kB 65.6 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp38-cp38-linux_x86_64.whl size=3825477 sha256=84eda66f05bf47c422c1b96bac6f60992f8e8ce23b916b67ef7dfa86f5a07986
  Stored in directory: /root/.cache/pip/wheels/00/ac/67/6f4536c819ed560c2c7e17c0f7a920e3e50c26108616087d05
  Created wheel for lightfm: filename=lightfm-1.16-cp38-cp38-linux_x86_64.whl size=746171 sha256=66c75b9ba528af03bd78cffb3c02ad3476f1430456ee8f0b962797dae595170e
  Store

In [2]:
import dill
import pandas as pd
import numpy as np
import scipy as sp
import requests
from tqdm.auto import tqdm
from scipy.stats import mode 
from implicit.nearest_neighbours import BM25Recommender
import warnings
warnings.filterwarnings("ignore")

from rectools import Columns
from rectools.dataset import Dataset
from rectools.models.popular import PopularModel 
from rectools.models.implicit_knn import ImplicitItemKNNWrapperModel

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)

# 🎬 Get KION dataset 

<a href="https://ods.ai/competitions/competition-recsys-21/data"> Dataset description [ru] </a>


In [3]:
# download dataset by chunks
url = "https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip"

req = requests.get(url, stream=True)

with open('kion_train.zip', "wb") as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download:   0%|          | 0.00/78.8M [00:00<?, ?iB/s]

In [4]:
!unzip kion_train.zip

Archive:  kion_train.zip
   creating: kion_train/
  inflating: kion_train/interactions.csv  
  inflating: __MACOSX/kion_train/._interactions.csv  
  inflating: kion_train/users.csv    
  inflating: __MACOSX/kion_train/._users.csv  
  inflating: kion_train/items.csv    
  inflating: __MACOSX/kion_train/._items.csv  


# EDA

In [5]:
interactions = pd.read_csv('kion_train/interactions.csv')
users = pd.read_csv('kion_train/users.csv')
items = pd.read_csv('kion_train/items.csv')

In [6]:
# rename columns, convert timestamp
interactions.rename(columns={'last_watch_dt': Columns.Datetime,
                            'total_dur': Columns.Weight}, 
                    inplace=True) 

interactions['datetime'] = pd.to_datetime(interactions['datetime'])

## interactions

In [7]:
pd.concat([interactions.head(), interactions.tail()])

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0
5476246,648596,12225,2021-08-13,76,0.0
5476247,546862,9673,2021-04-13,2308,49.0
5476248,697262,15297,2021-08-20,18307,63.0
5476249,384202,16197,2021-04-19,6203,100.0
5476250,319709,4436,2021-08-15,3921,45.0


In [8]:
print(f"Interactions dataframe shape: {interactions.shape}")
print(f"Unique users in interactions: {interactions['user_id'].nunique():_}")
print(f"Unique items in interactions: {interactions['item_id'].nunique():_}")

Interactions dataframe shape: (5476251, 5)
Unique users in interactions: 962_179
Unique items in interactions: 15_706


In [9]:
max_date = interactions['datetime'].max()
min_date = interactions['datetime'].min()

print(f"min date in interactions: {min_date}")
print(f"max date in interactions: {max_date}")

min date in interactions: 2021-03-13 00:00:00
max date in interactions: 2021-08-22 00:00:00


In [10]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5476251 entries, 0 to 5476250
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   user_id      int64         
 1   item_id      int64         
 2   datetime     datetime64[ns]
 3   weight       int64         
 4   watched_pct  float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 208.9 MB


## users

In [11]:
pd.concat([users.head(), users.tail()])

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
840192,339025,age_65_inf,income_0_20,Ж,0
840193,983617,age_18_24,income_20_40,Ж,1
840194,251008,NaN,NaN,NaN,0
840195,590706,NaN,NaN,Ж,0
840196,166555,age_65_inf,income_20_40,Ж,0


In [12]:
print(f"Users dataframe shape {users.shape}")
print(f"Unique users: {users['user_id'].nunique():_}")

Users dataframe shape (840197, 5)
Unique users: 840_197


## items

In [13]:
pd.concat([items.head(3), items.tail(3)])

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Грандинетти, Джеральдин Чаплин, Елена Анайя, Каэтано Велозо, Леонор Уотлинг, Лола Дуэньяс, Лолес Леон, Малу Айродо, Мариола Фуэнтес, Пас Вега, Пина Бауш, Ро...",Мелодрама легендарного Педро Альмодовара «Поговори с ней» в 2003 году получила премию «Оскар» за лучший сценарий. Журналист Марко берет интервью у знаменитой женщины-тореро Лидии и вскоре влюбляе...,"Поговори, ней, 2002, Испания, друзья, любовь, сильные, женщины, преодоление, трудностей, отношения, дружба, отношения, паре, отношения, мужчины, женщины, романтические, отношения, потеря, близких,..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон Манцукас, Джон Глейсер, Карл Грин, Кристен Риттер, Лэнс Реддик, Морис Комт, Патрик Кернс, Ребекка Коллинз, Роза Салазар, Росс П. Кук, Стеффи Гроут, Ти...","Уморительная современная комедия на популярную тему о том, как не надо отмечать мальчишник. Главный герой усвоил, что не надо звать на свадьбу своего друга Джейсона, из-за которого он вместо сваде...","Голые, перцы, 2014, США, друзья, свадьбы, преодоление, трудностей, расставания, отношения, дружба, риск, недоразумение, мужская, дружба, мальчишники, девичники"
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман, Дэн Риззуто, Кендес Илэйн Калтраро, Кит Джардин, Лекса Дойг, Майкл Джей Уайт, Майкл Шэнкс, Майкл Эклунд, Питер Брайант, Питер Кент, Стив Бачич, Стив ...","Профессиональный рестлер Стив Остин («Все или ничего») и темнокожий мачо Майкл Джей Уайт («Темный рыцарь») в интригующем криминальном боевике. В центре сюжета – команда спецназовцев, которая оказа...","Тактическая, сила, 2011, Канада, бандиты, гангстеры, преступления, преодоление, трудностей, убийства, убийцы, настоящие, мужчины, риск, недоразумение, силы, правопорядка, борьба, за, выживание, сп..."
15960,10632,series,Сговор,Hassel,2017.0,"драмы, триллеры, криминал",Россия,0.0,18.0,NaN,"Эшреф Рейбрук, Амир Камдин, Эрик Эгер","Ола Рапас, Алиетт Офейм, Уильма Лиден, Шанти Рони, Тома Холмин","Криминальная драма по мотивам романов о шведском детективе Роланде Хасселе. Средь бела дня убит полицейский, и нити в этом деле ведут прямо в коридоры власти. Расследованием занимается детектив Ха...","Сговор, 2017, Россия"
15961,4538,series,Среди камней,Darklands,2019.0,"драмы, спорт, криминал",Россия,0.0,18.0,NaN,"Марк О’Коннор, Конор МакМахон","Дэйн Уайт О’Хара, Томас Кэйн-Бирн, Джудит Родди, Марк О’Халлоран, Джимми Смоллхорн","Семнадцатилетний Дэмиен мечтает вырваться за пределы своего района и стать профессиональным бойцом. Когда его кумир и старший брат исчезает, парень попадает в чуждый ему мир насилия, наркотиков и ...","Среди, камней, 2019, Россия"
15962,3206,series,Гоша,NaN,2019.0,комедии,Россия,0.0,16.0,NaN,Михаил Миронов,"Мкртыч Арзуманян, Виктория Рунцова","Добродушный Гоша не может выйти из дома, чтобы не попасть в нелепую и курьёзную историю. Но даже неудачники мечтают о любви, и наш герой — не исключение, ведь оптимизма ему не занимать.","Гоша, 2019, Россия"


In [14]:
print(f"Items dataframe shape {items.shape}")
print(f"Unique item_id: {items['item_id'].nunique():_}")

Items dataframe shape (15963, 14)
Unique item_id: 15_963


# Popular model

Модель популярного за 2 последние недели в interactions.csv:

In [15]:
timestamp = pd.Timestamp('2021-08-08')
df = interactions[interactions.datetime > timestamp]

dataset = Dataset.construct(
    interactions_df=df,
    user_features_df=None,
    item_features_df=None
)

In [16]:
pop = PopularModel()
pop.fit(dataset);

Обучаем модель и формируем список айтемов:

In [17]:
pop_recos = pop.recommend(
    np.array([1]), 
    dataset=dataset, 
    k=10, 
    filter_viewed=False
)

In [18]:
pop_recos = pop_recos['item_id'].to_list()
pop_recos

[10440, 9728, 15297, 13865, 12192, 341, 4151, 3734, 12360, 7793]

Сохраняем список для дальнейшего использования на холодных юзерах:

In [19]:
with open('popular_model.dill', 'wb') as f:
  dill.dump(pop_recos, f)

# BM25Recommender

Подготавливаем датасет:

In [20]:
_, bins = pd.qcut(items["release_year"], 10, retbins=True)
labels = bins[:-1]

year_feature = pd.DataFrame(
    {
        "id": items["item_id"],
        "value": pd.cut(items["release_year"], bins=bins, labels=labels),
        "feature": "release_year",
    }
)

items["genre"] = items["genres"].str.split(",")
items[["genre", "genres"]].head(3)

genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"

item_feat = pd.concat([genre_feature, year_feature])
item_feat = item_feat[item_feat['id'].isin(interactions['item_id'])]

In [21]:
bm25model = ImplicitItemKNNWrapperModel(BM25Recommender(K=10))

In [22]:
dataset = Dataset.construct(
      interactions_df=interactions,
      user_features_df=None,
      item_features_df=item_feat,
      cat_item_features=['genre', 'release_year']
)

Обучаем модель и делаем предсказания:

In [23]:
bm25model.fit(dataset);

In [24]:
recos = bm25model.recommend(
    interactions.user_id.unique(),
    dataset=dataset,
    k=10,
    filter_viewed=True
)

In [25]:
recos

,user_id,item_id,score,rank
0,176549,13865,8.446965e+10,1
1,176549,10440,7.295995e+10,2
2,176549,15297,6.384488e+10,3
3,176549,3734,6.199872e+10,4
4,176549,4151,2.759679e+10,5
...,...,...,...,...
9154676,697262,9728,9.115432e+10,5
9154677,697262,142,7.403072e+10,6
9154678,697262,2657,7.134247e+10,7
9154679,697262,12192,6.349703e+10,8


Формируем словарь для быстрой выдачи предсказаний в дальнейшем:

In [26]:
df = recos[['user_id', 'item_id']].set_index('user_id')
df

,item_id
user_id,
176549,13865
176549,10440
176549,15297
176549,3734
176549,4151
...,...
697262,9728
697262,142
697262,2657


In [27]:
df_dict = df.groupby('user_id').agg({'item_id': list}).to_dict()
df_dict

{'item_id': {0: [10440,
   4151,
   13865,
   3734,
   142,
   2657,
   4880,
   12995,
   14431,
   8636],
  1: [15297, 13865, 9728, 4151, 2657, 3734, 142, 9996, 4880, 12192],
  2: [9728, 13865, 3734, 10440, 15297, 3182, 7626, 12173, 14317, 4151],
  3: [15297, 13865, 9996, 8636, 14431, 7571, 16166, 3182, 7829, 14317],
  4: [9728, 13865, 10440, 15297, 8636, 3734, 4151, 4457, 142, 9900],
  5: [5324, 696, 7653, 12314, 9785, 14302, 6902, 1404, 13995, 7248],
  7: [10440, 15297, 4880, 13865, 16228, 4151, 805, 14266, 9996],
  8: [15297, 10440, 4151, 9728, 13865, 142, 4880, 3734, 4740],
  9: [4880, 12192, 12995, 8636, 6809],
  10: [10440, 9728, 15297, 3734, 4151, 4880, 2657, 142, 12995, 16228],
  11: [10440, 4151, 13865, 9728, 3734, 142, 2657, 4880, 9996, 8636],
  12: [10440, 15297, 16228, 2657, 4151, 6382, 15399, 3734, 142],
  13: [10440, 15297, 13865, 3734, 9728, 142, 2657, 9996, 7417, 849],
  14: [15297, 13865, 9728, 4151, 2657, 3734, 142, 9996, 4880, 16228],
  15: [10440, 4151, 3734, 9728

Сохраняем словарь:

In [28]:
with open('bm25userknn.dill', 'wb') as f:
    dill.dump(df_dict, f)